# CX 4230, Spring 2016: [18] Discrete event simulation of a gas station

Recall the introduction to queueing models and discrete event simulators from the last class: [link](https://t-square.gatech.edu/access/content/group/gtc-59b8-dc03-5a67-a5f4-88b8e4d5b69a/cx4230-sp16--17-queueing.pdf). In this notebook, you will implement it.

## Exponential random numbers

Recall that in a queueing model, it is common to assume that customer interarrival times and service times are independent and identically distributed random variables. Classically, the most commonly assumed distribution is _exponential_.

More specifically, an exponentially distributed random variable $X \sim \mathcal{E}(\lambda)$ has the probability density function,

$$
  f_X(x) = \lambda \cdot \exp\left(-\frac{x}{\lambda}\right),
$$

where $\lambda$ is the mean of the distribution.

Using Numpy, these are easy to generate using the function, `numpy.random.exponential()`: http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.random.exponential.html.

Here is a quick demo.

In [1]:
from numpy.random import exponential

X_MEAN = 10.0
X_COUNT = 5
x_values = exponential (X_MEAN, X_COUNT)

print ("X ~ Exp(%g):" % X_MEAN)
for (i, x_i) in enumerate (x_values):
    print ("  X_%d = %g" % (i, x_i))

X ~ Exp(10):
  X_0 = 8.9271
  X_1 = 7.13314
  X_2 = 0.407493
  X_3 = 0.639331
  X_4 = 6.1763


As a sanity check, let's generate a large number of values and compare the sample mean to the desired (true) mean.

In [2]:
from numpy import mean

# @Demo
N_BIG = 1000
big_mean = mean (exponential (X_MEAN, N_BIG))
print ("\nSample mean of %d values: %g" % (N_BIG, big_mean))


Sample mean of 1000 values: 9.80182


## Priority queues

To maintain the future event list, you need some kind of priority queue data structure. One classical choice is to use a heap, for which there is a standard implementation in Python: [link](http://www.bogotobogo.com/python/python_PriorityQueue_heapq_Data_Structure.php)

Here's a quick demo.

In [3]:
from heapq import heappush, heappop, heapify

In [4]:
# Method 1: Convert any Python list into a heap
h1 = list (x_values)
print ("Initial values:", h1)

heapify (h1)
print ("\nHeapified:", h1)

print ("\nExtracting mins...")
for i in range (len (h1)):
    print (i, ":", heappop (h1))

Initial values: [8.9271038997653154, 7.1331379790454204, 0.40749271779790519, 0.63933145928678492, 6.1763008721198993]

Heapified: [0.40749271779790519, 0.63933145928678492, 8.9271038997653154, 7.1331379790454204, 6.1763008721198993]

Extracting mins...
0 : 0.407492717798
1 : 0.639331459287
2 : 6.17630087212
3 : 7.13313797905
4 : 8.92710389977


In [5]:
# Method 2: Insert values into the heap one at a time

print ("Inserting...")
h2 = []
for (i, x_i) in enumerate (x_values):
    print (i, ":", x_i)
    heappush (h2, x_i)
    
print ("\nHeap:", h2)
    
print ("\nExtracting minima...")
for i in range (len (h2)):
    print (i, ":", heappop (h2))

Inserting...
0 : 8.92710389977
1 : 7.13313797905
2 : 0.407492717798
3 : 0.639331459287
4 : 6.17630087212

Heap: [0.40749271779790519, 0.63933145928678492, 7.1331379790454204, 8.9271038997653154, 6.1763008721198993]

Extracting minima...
0 : 0.407492717798
1 : 0.639331459287
2 : 6.17630087212
3 : 7.13313797905
4 : 8.92710389977


## A generic discrete event simulation engine

We can build a simple, generic discrete event simulation engine. This engine manages the future event list, which you'll recall is a priority queue of timestamped events. It continually removes the event with the lowest timestamp and processes it.

Suppose we represent an event by a tuple, `(t, e)`, where `t` is the event's timestamp and `e` is an event handler. An event handler is simply a function. Let's suppose that this function takes two arguments, `e (t, s)`, where `t` is (again) the timestamp and `s` is the system state, encoded in an application-specific way. When `e (t, s)` executes, it may update the state `s`.

**Exercise.** Complete the following function, which implements a generic discrete event simulation engine. The future events list is `events`. The initial system state is `initial_state`; the starter code below makes a copy of this state as a variable `s`, which your simulator can modify. Additionally, you should correct the last `print()` statement so that instead of a pair of `None` values it prints the timestamp and event name (name of the event handler).

In [6]:
from copy import deepcopy

def simulate (events, initial_state):
    s = deepcopy (initial_state)
    
    print ("\nFuture event list:\n%s" % str (events))
    print ("\nt=0: %s" % str (s))
        
    while events:
        # @YOUSE: Get event and process it
        (t, e) = heappop(events)
        e(t, s)
        
        print ("t=%d: event '%s' => '%s'" % (t, e, str (s)))

## Instantiating the simulator

For the gas station model, we asked you to assume the interarrival times, pumping times, and shopping times were exponential i.i.d. random variables. So, let's start by defining some parameters for these distributions. Let's also pre-generate some number of car arrivals.

In [7]:
# Event parameters
MEAN_INTERARRIVAL_TIME = 15.0 # minutes
MEAN_PUMPING_TIME = 5.0 # minutes
MEAN_SHOPPING_TIME = 10.0 # minutes

# Number of customers (cars)
NUM_CARS = 5

# Pre-generate some interarrival times
car_interarrival_times = exponential (MEAN_INTERARRIVAL_TIME, NUM_CARS)
print ("Interarrival times (in minutes) of all cars:\n", car_interarrival_times)

Interrival times (in minutes) of all cars:
 [ 51.318302     2.69856438  22.07303371  20.42265916  14.64580401]


Recall that the state consists of the logical simulation time (`now`) and three state variables: `AtPump`, `AtStore`, and `PumpFree`. Let's create this state.

In [8]:
state = {'AtPump': 0          # no. cars at pump or waiting
         , 'AtStore': 0       # no. cars at store
         , 'PumpFree': True   # True <==> pump is available
        }

Let's represent an _event_ as a tuple, `(t, e)`, where `t` is the timestamp of the event and `e` is an event handler, implemented as a Python function.

If the future event list is stored in a global priority queue called `events`, the following function will insert an event into that queue.

In [9]:
def schedule (t, e):
    """
    Schedules a new event `e` at time `t`.
    """
    global events
    print ("  ==> '%s' @ t=%g" % (e.__name__, t))
    heappush (events, (t, e))

**Exercise.** Implement an event handler to process a car arrival event. Assume that event handlers take as input the timestamp `t` of the event and the state `s` of the system at time `t`.

In [10]:
def arrives (t, s):
    """
    Processes an arrival event at time `t` for a system in state `s`.
    Schedules a pumping event if the pump is free. Returns the new
    system state.
    """
    # @YOUSE
    s['AtPump'] += 1
    if(s["PumpFree"]):
        s["PumpFree"] = False
        t_done = t + exponential(MEAN_PUMPING_TIME)
        schedule(t_done, finishes)
    return s

**Exercise.** Implement a function to process the event for a car that finishes pumping gas.

In [11]:
def finishes (t, s):
    """
    Processes a finished-pumping event at time `t` for a system in
    state `s`. Schedules a pumping event if any cars are waiting.
    Returns the new system state.
    """
    # @YOUSE
    s['AtPump'] -= 1
    if s['AtPump']:
        schedule( t+exponential(MEAN_PUMPING_TIME), finishes)
    else:
        s['PumpFree'] = True
    
    s['AtStore'] += 1
    t_done = t + exponential(MEAN_SHOPPING_TIME)
    schedule(t_done, departs)
    return s

**Exercise.** Implement a function to process the event for a car that leaves the store.

In [12]:
def departs (t, s):
    """
    Processes a departure from the station event at
    time `t` for a system in state `s`.
    """
    # @YOUSE
    s['AtStore'] -= 1
    
    return s

**Exercise.** Create an initial future events list by converting the raw interarrival times into arrival events and inserting them into the future events list.

In [13]:
# Hint: This function may prove useful
from numpy import cumsum

events = []  # Future event list, initially empty
t = 0
# @YOUSE: Create initial events from all car arrivals
for i in range(NUM_CARS):
    schedule(t, arrives)
    t += exponential(MEAN_INTERARRIVAL_TIME)
# Test code
print ("\nContents of `events[:]`:")
for (i, event) in enumerate (events):
    print ("[%d] t=%g: %s" % (i, event[0], event[1].__name__))

  ==> 'arrives' @ t=0
  ==> 'arrives' @ t=28.2718
  ==> 'arrives' @ t=63.0858
  ==> 'arrives' @ t=67.2148
  ==> 'arrives' @ t=75.3158

Contents of `events[:]`:
[0] t=0: arrives
[1] t=28.2718: arrives
[2] t=63.0858: arrives
[3] t=67.2148: arrives
[4] t=75.3158: arrives


In [14]:
# More test code: If everything worked, so should this simulation!
simulate (events, state)


Future event list:
[(0, <function arrives at 0x107ce9378>), (28.271821310326278, <function arrives at 0x107ce9378>), (63.085768316839676, <function arrives at 0x107ce9378>), (67.21484123390591, <function arrives at 0x107ce9378>), (75.31581049394012, <function arrives at 0x107ce9378>)]

t=0: {'AtPump': 0, 'AtStore': 0, 'PumpFree': True}
  ==> 'finishes' @ t=2.9604
t=0: event '<function arrives at 0x107ce9378>' => '{'AtPump': 1, 'AtStore': 0, 'PumpFree': False}'
  ==> 'departs' @ t=4.12722
t=2: event '<function finishes at 0x107ce9950>' => '{'AtPump': 0, 'AtStore': 1, 'PumpFree': True}'
t=4: event '<function departs at 0x107ce9bf8>' => '{'AtPump': 0, 'AtStore': 0, 'PumpFree': True}'
  ==> 'finishes' @ t=37.0822
t=28: event '<function arrives at 0x107ce9378>' => '{'AtPump': 1, 'AtStore': 0, 'PumpFree': False}'
  ==> 'departs' @ t=51.355
t=37: event '<function finishes at 0x107ce9950>' => '{'AtPump': 0, 'AtStore': 1, 'PumpFree': True}'
t=51: event '<function departs at 0x107ce9bf8>' => '{